<a href="https://colab.research.google.com/github/raleight1/mgmt467-analytics-portfolio/blob/main/Labs/Labs%201-3/Lab_VertexAI_BigQuery_PromptsOnly.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab: Vertex AI–Assisted BigQuery Analytics — Example Prompts
**Goal:** Practice moving from simple SQL to complex analytics in BigQuery using *only* carefully engineered prompts with Vertex AI (Gemini).  
**Important:** This notebook contains **prompts only** (no starter code). Paste the prompts into **Vertex AI Studio**, **Vertex AI in Colab Enterprise**, or your chosen chat interface, and then run the generated SQL directly in **BigQuery**. If you decide to automate later, you can ask Vertex AI to convert the winning SQL into a Colab pipeline.

## How to use this prompts-only notebook
1. Open **Vertex AI Studio** (or Gemini in Colab Enterprise chat panel).  
2. Copy a prompt from this notebook and paste it into the model. Do **not** paste any code from here; let the model generate it.  
3. Run the generated SQL in **BigQuery** (Console → BigQuery Studio).  
4. Iterate: refine the prompt when results aren’t what you expect.  
5. Document: capture your final SQL, plus a one-sentence takeaway, in your notes/README.

## Dataset assumptions
Use one of these sources (adjust table paths accordingly):
- **Global Superstore (Kaggle)** loaded into BigQuery (e.g., `[YOUR_PROJECT].superstore_data.sales`)  
- **TheLook eCommerce** public dataset: `bigquery-public-data.thelook_ecommerce`  
If you are using *Global Superstore*, make sure column names match your schema (e.g., `Order_Date`, `Region`, `Category`, `Sub_Category`, `Sales`, `Profit`, `Discount`, `State`, `Customer_ID`, `Ship_Mode`).

---
## Prompting guardrails (quick checklist)
- **Be explicit**: table path, column names, filters, output columns, sort order, and limits.  
- **Ask for runnable SQL**: “Return a BigQuery SQL block only.”  
- **Control cost**: ask for `LIMIT` during exploration and remove it for the final run.  
- **Validate**: request a brief explanation of why each clause is present and how you can sanity-check results.
---

## Install Dependencies

In [ ]:
# Install the Google Cloud BigQuery client library
!pip install google-cloud-bigquery==3.17.0 pandas==2.1.4

# Authenticate your Colab environment
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 230.2/230.2 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 67.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 64.3 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.2
    Uninstalling pandas-2.2.2:
      Successfully uninstalled pandas-2.2.2
  Attempting uninstall: google-cloud-bigquery
    Found existing installation: google-cloud-bigquery 3.35.1
    Uninstalling google-cloud-bigquery-3.35.1:
      Successfully uninstalled google-cloud-bigquery-3.35.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency co

Authenticated


## Copy Schema to a dataframe

In [ ]:
from google.cloud import bigquery
import pandas as pd

# Replace with your Google Cloud Project ID
project_id = 'mgmt-467-lab-1-ai-assisted-sql' # This is derived from your provided table name
dataset_id = 'lab1_foundation'
table_id = 'superstore'

# Construct a BigQuery client object.
client = bigquery.Client(project=project_id)

# Get the table object
table_ref = client.dataset(dataset_id).table(table_id)
table = client.get_table(table_ref)

# Extract schema information
schema_list = []
for field in table.schema:
    schema_list.append({
        'name': field.name,
        'field_type': field.field_type,
        'mode': field.mode,
        'description': field.description
    })

# Convert to Pandas DataFrame
schema_df = pd.DataFrame(schema_list)

# Display the schema DataFrame (optional, for verification)
print("Schema DataFrame created:")
# To see the output, run the code.


Schema DataFrame created:


## CLean Column Names

In [ ]:
# --- 1. Clean the Column Names ---
# Create a 'clean_name' column with standard naming conventions:
# lowercase, with spaces and hyphens replaced by underscores.
schema_df['clean_name'] = schema_df['name'].str.lower().str.replace(' ', '_').str.replace('-', '_')


# --- 2. Generate the Aliases for the SELECT Clause ---
column_expressions = []
for index, row in schema_df.iterrows():
    original_name = row['name']
    clean_name = row['clean_name']

    # If the original name contains a space or special character, it needs to be
    # enclosed in backticks (`) in the SQL statement.
    if ' ' in original_name or '-' in original_name:
        expression = f'`{original_name}` AS {clean_name}'
    else:
        # If the name is already clean, we still alias it for consistency.
        expression = f'{original_name} AS {clean_name}'
    column_expressions.append(expression)

# Join all the individual column expressions into a single, formatted string.
select_clause = ",\n  ".join(column_expressions)


# --- 3. Construct the Final CREATE VIEW Statement ---
new_view_id = 'superstore_clean' # You can change this if you like

create_view_sql = f"""
CREATE OR REPLACE VIEW `{project_id}.{dataset_id}.{new_view_id}` AS
SELECT
  {select_clause}
FROM
  `{project_id}.{dataset_id}.{table_id}`;
"""

# --- 4. Print the Final SQL ---
print("--- Copy the SQL below and run it in your BigQuery Console ---")
print(create_view_sql)

--- Copy the SQL below and run it in your BigQuery Console ---

CREATE OR REPLACE VIEW `mgmt-467-lab-1-ai-assisted-sql.lab1_foundation.superstore_clean` AS
SELECT
  `Row ID` AS row_id,
  `Order ID` AS order_id,
  `Order Date` AS order_date,
  `Ship Date` AS ship_date,
  `Ship Mode` AS ship_mode,
  `Customer ID` AS customer_id,
  `Customer Name` AS customer_name,
  Segment AS segment,
  Country AS country,
  City AS city,
  State AS state,
  `Postal Code` AS postal_code,
  Region AS region,
  `Product ID` AS product_id,
  Category AS category,
  `Sub-Category` AS sub_category,
  `Product Name` AS product_name,
  Sales AS sales,
  Quantity AS quantity,
  Discount AS discount,
  Profit AS profit
FROM
  `mgmt-467-lab-1-ai-assisted-sql.lab1_foundation.superstore`;



## Generate View with standard column naming convention

In [ ]:
# Execute the CREATE VIEW SQL query
try:
    query_job = client.query(create_view_sql)  # API request
    query_job.result()  # Waits for the query to finish
    print(f"View '{new_view_id}' created/replaced successfully in dataset '{dataset_id}'.")
except Exception as e:
    print(f"An error occurred while creating the view: {e}")

# Now, let's print 10 rows from the newly created view to verify
print(f"\n--- First 10 rows from the new view '{new_view_id}' ---")
try:
    # Construct a reference to the new view
    view_table_ref = client.dataset(dataset_id).table(new_view_id)

    # Fetch the first 10 rows
    rows = client.list_rows(view_table_ref, max_results=10)

    # Print header
    print(" | ".join([field.name for field in rows.schema]))
    print("-" * 80) # Separator

    # Print rows
    for row in rows:
        print(" | ".join([str(item) for item in row.values()]))

except Exception as e:
    print(f"An error occurred while fetching rows from the view: {e}")



View 'superstore_clean' created/replaced successfully in dataset 'lab1_foundation'.

--- First 10 rows from the new view 'superstore_clean' ---
row_id | order_id | order_date | ship_date | ship_mode | customer_id | customer_name | segment | country | city | state | postal_code | region | product_id | category | sub_category | product_name | sales | quantity | discount | profit
--------------------------------------------------------------------------------
An error occurred while fetching rows from the view: 400 GET https://bigquery.googleapis.com/bigquery/v2/projects/mgmt-467-lab-1-ai-assisted-sql/datasets/lab1_foundation/tables/superstore_clean/data?maxResults=10&formatOptions.useInt64Timestamp=True&prettyPrint=false: Cannot list a table of type VIEW.


In [ ]:
# This assumes your 'client' object from the previous cell is still active
# and correctly authenticated.

print("✅ Step 1: Defining the query string...")

query_string = """
SELECT
  order_id,
  customer_name,
  product_name,
  sales,
  profit
FROM
  `mgmt-467-lab-1-ai-assisted-sql.lab1_foundation.superstore_clean`
LIMIT 10;
"""

print("✅ Step 2: Sending the query to BigQuery. This may take a moment...")

# Use a try-except block to catch potential errors
try:
    query_job = client.query(query_string)

    print("✅ Step 3: Waiting for query to complete and fetching results...")
    results_df = query_job.to_dataframe()

    print(f"✅ Step 4: Query finished. Found {len(results_df)} rows.")

    if results_df.empty:
        print("\n⚠️ The query ran successfully but returned an empty result. Please double-check that your 'superstore_clean' view exists and the original table has data.")
    else:
        print("\n--- Displaying Results ---")
        display(results_df)

except Exception as e:
    print(f"\n❌ An error occurred: {e}")

✅ Step 1: Defining the query string...
✅ Step 2: Sending the query to BigQuery. This may take a moment...
✅ Step 3: Waiting for query to complete and fetching results...
✅ Step 4: Query finished. Found 10 rows.

--- Displaying Results ---


,order_id,customer_name,product_name,sales,profit
0,CA-2015-154900,Sung Shariari,Avery 518,3.15,1.5120
1,CA-2015-154900,Sung Shariari,Adams Telephone Message Book W/Dividers/Space ...,22.72,10.2240
2,US-2016-152415,Patrick O'Donnell,"C-Line Magnetic Cubicle Keepers, Clear Polypro...",14.82,6.2244
3,US-2016-152415,Patrick O'Donnell,"Howard Miller 14-1/2"" Diameter Chrome Round Wa...",191.82,61.3824
4,CA-2016-153269,Pamela Stobb,"Personal Folder Holder, Ebony",11.21,3.3630
5,CA-2016-153269,Pamela Stobb,"Situations Contoured Folding Chairs, 4/Set",354.90,88.7250
6,CA-2016-153269,Pamela Stobb,Xerox 193,17.94,8.7906
7,CA-2016-153269,Pamela Stobb,GBC Binding covers,51.80,23.3100
8,CA-2015-158792,Brian Dahlen,Staples,22.20,10.4340
9,CA-2016-141082,Fred McMath,Avery 517,3.69,1.7343


## Part A — SQL Warm‑Up (SELECT, WHERE, ORDER BY, LIMIT, DISTINCT)
**Aim:** Build confidence with precise, unambiguous prompts that yield clean, runnable SQL.

### A1. Unique values (DISTINCT)
**Prompt (paste in Vertex AI):**
```
Act as a senior BigQuery analyst. Produce a **single runnable BigQuery SQL** (no commentary) for:
- Task: List all unique `Sub_Category` values sold in the 'West' region.
- Table: `mgmt-467-lab-1-ai-assisted-sql.lab1_foundation.superstore_clean`
- Filter: `Region = 'West'`
- Output: a single column named `Sub_Category`
- Sort: alphabetically A→Z
- Add: `LIMIT 100` to control cost during exploration.
```
**Reflection:** Did the result match your expectations? If not, what ambiguity in your prompt might have caused the mismatch?

The result followed my basic expectations and printed out the different sub categories in descending order.

In [ ]:
query_string = """
SELECT
  DISTINCT sub_category
FROM
  `mgmt-467-lab-1-ai-assisted-sql.lab1_foundation.superstore_clean`
WHERE
  Region = 'West'
ORDER BY sub_category
LIMIT 100
"""
query_job = client.query(query_string)
results_df = query_job.to_dataframe()
display(results_df)

,sub_category
0,Accessories
1,Appliances
2,Art
3,Binders
4,Bookcases
5,Chairs
6,Copiers
7,Envelopes
8,Fasteners
9,Furnishings


### A2. Top‑N by metric (ORDER BY … DESC)
**Prompt:**
```
BigQuery SQL only.
Task: Return the top 10 customers by total profit.
Table: `mgmt-467-lab-1-ai-assisted-sql.lab_foundation.superstore_clean`
Columns used: `customer_id`, `profit`
Output columns: `Customer_ID`, `total_profit`
Logic: SUM Profit per customer, order by `total_profit` DESC
Add `LIMIT 10`.
```
**Tip:** If your schema uses different identifiers (e.g., `Customer Name`), restate column names explicitly.

In [ ]:
query_string = """
SELECT
    customer_id AS Customer_ID,
    SUM(profit) AS total_profit
FROM
    `mgmt-467-lab-1-ai-assisted-sql.lab1_foundation.superstore_clean`
GROUP BY
    customer_id
ORDER BY
    total_profit DESC
LIMIT 10;
"""
query_job = client.query(query_string)
results_df = query_job.to_dataframe()
display(results_df)

,Customer_ID,total_profit
0,TC-20980,8981.3239
1,RB-19360,6976.0959
2,SC-20095,5757.4119
3,HL-15040,5622.4292
4,AB-10105,5444.8055
5,TA-21385,4703.7883
6,CM-12385,3899.8904
7,KD-16495,3038.6254
8,AR-10540,2884.6208
9,DR-12940,2869.0760


### A3. Basic filtering (WHERE) + sanity checks
**Prompt:**
```
BigQuery SQL only.
Task: Count orders shipped with each `Ship_Mode`, but only for orders in the 'Technology' category.
Table: `[YOUR_PROJECT].superstore_data.sales`
Output: `Ship_Mode`, `order_count`
Logic: COUNT(*) grouped by `Ship_Mode`
Sort by `order_count` DESC
```
**Validation ask:** “Also list two quick sanity checks to verify the numbers.”

In [ ]:
query_string = """
SELECT
    ship_mode,
    COUNT(*) AS order_count
FROM
    `mgmt-467-lab-1-ai-assisted-sql.lab1_foundation.superstore_clean`
WHERE
    category = 'Technology'
GROUP BY
    ship_mode
ORDER BY
    order_count DESC;
"""
query_job = client.query(query_string)
results_df = query_job.to_dataframe()
display(results_df)

,ship_mode,order_count
0,Standard Class,1082
1,Second Class,366
2,First Class,301
3,Same Day,98


In [ ]:
#Sanity Check 1
query_string = """SELECT
  COUNT(*) AS total_technology_orders
FROM
  `mgmt-467-lab-1-ai-assisted-sql.lab1_foundation.superstore_clean`
WHERE
  category = 'Technology';"""

query_job = client.query(query_string)
results_df = query_job.to_dataframe()
display(results_df)

,total_technology_orders
0,1847


## Part B — Grouped Analytics (GROUP BY, HAVING)
**Aim:** Turn raw facts into grouped metrics and filtered aggregations.

### B1. KPI aggregation with WHERE + GROUP BY
**Prompt:**
```
BigQuery SQL only.
Task: Compute monthly revenue for the last 12 full months.
Table: `[YOUR_PROJECT].superstore_data.sales`
Assume: `Order_Date` is a DATE or TIMESTAMP column named exactly `Order_Date`.
Output: `year_month` (YYYY-MM format), `monthly_revenue`
Logic: Truncate date to month, SUM `Sales`, filter to last 12 full months.
Sort by `year_month` ascending.
Include a `LIMIT` safeguard for exploration.
```

In [ ]:
query_string = """
SELECT
  FORMAT_DATE('%Y-%m', DATE_TRUNC(order_date, MONTH)) AS year_month,
  SUM(sales) AS monthly_revenue
FROM
  `mgmt-467-lab-1-ai-assisted-sql.lab1_foundation.superstore_clean`
WHERE
  -- Filter for the 12 full months preceding the month of the latest order
  order_date >= DATE_SUB(
    (SELECT DATE_TRUNC(MAX(order_date), MONTH) FROM `mgmt-467-lab-1-ai-assisted-sql.lab1_foundation.superstore_clean`),
    INTERVAL 12 MONTH
  )
  AND order_date < (SELECT DATE_TRUNC(MAX(order_date), MONTH) FROM `mgmt-467-lab-1-ai-assisted-sql.lab1_foundation.superstore_clean`)
GROUP BY
  year_month
ORDER BY
  year_month ASC
LIMIT 100
"""
query_job = client.query(query_string)
results_df = query_job.to_dataframe()
display(results_df)

,year_month,monthly_revenue
0,2016-12,96999.0430
1,2017-01,43971.3740
2,2017-02,20301.1334
3,2017-03,58872.3528
4,2017-04,36521.5361
5,2017-05,44261.1102
6,2017-06,52981.7257
7,2017-07,45264.4160
8,2017-08,63120.8880
9,2017-09,87866.6520


### B2. Post‑aggregation filter (HAVING)
**Prompt:**
```
BigQuery SQL only.
Task: Find sub-categories whose total profit over the entire dataset is negative.
Table: `[YOUR_PROJECT].superstore_data.sales`
Output: `Sub_Category`, `total_profit`
Logic: SUM `Profit` GROUP BY `Sub_Category`, HAVING SUM(Profit) < 0
Sort by `total_profit` ASC (most negative first).
```
**Why HAVING?** Ask the model to include a 1-sentence explanation of why HAVING is used instead of WHERE here.

The HAVING clause is used because it filters groups of rows after they have been aggregated by the GROUP BY clause, while the WHERE clause filters individual rows before the aggregation takes place.

In [ ]:
query_string = """
SELECT
    sub_category,
    SUM(profit) AS total_profit
FROM
    `mgmt-467-lab-1-ai-assisted-sql.lab1_foundation.superstore_clean`
GROUP BY
    sub_category
HAVING
    SUM(profit) < 0
ORDER BY
    total_profit ASC;
"""
query_job = client.query(query_string)
results_df = query_job.to_dataframe()
display(results_df)

,sub_category,total_profit
0,Tables,-17725.4811
1,Bookcases,-3472.5560
2,Supplies,-1189.0995


## Part C — Joins (dimension enrichment)
**Aim:** Use joins to enhance facts with attributes.

### C1. Join facts to a small dimension
*(If you have a customer or product dimension in your schema, use it. Otherwise, request a synthetic example.)*  
**Prompt:**
```
BigQuery SQL only.
Task: Join the sales table to a product dimension to report `Product_ID`, `Product_Name`, and total sales.
Tables: `[YOUR_PROJECT].superstore_data.sales` as s, `[YOUR_PROJECT].superstore_data.products` as p
Join key: `s.Product_ID = p.Product_ID`
Output: `Product_ID`, `Product_Name`, `total_sales`
Sort by `total_sales` DESC
```
**If you lack a dimension table:** Ask the model how to simulate one temporarily via a CTE.

In [ ]:
query_string = """
WITH products AS (
  -- This CTE simulates a product dimension table by selecting the unique
  -- product IDs and names from the main table.
  SELECT DISTINCT
    product_id,
    product_name
  FROM
    `mgmt-467-lab-1-ai-assisted-sql.lab1_foundation.superstore_clean`
)
SELECT
  p.product_id,
  p.product_name,
  SUM(s.sales) AS total_sales
FROM
  `mgmt-467-lab-1-ai-assisted-sql.lab1_foundation.superstore_clean` AS s
JOIN
  products AS p
  ON s.product_id = p.product_id
GROUP BY
  p.product_id,
  p.product_name
ORDER BY
  total_sales DESC
LIMIT 20;
"""
query_job = client.query(query_string)
results_df = query_job.to_dataframe()
display(results_df)

,product_id,product_name,total_sales
0,TEC-CO-10004722,Canon imageCLASS 2200 Advanced Copier,61599.8240
1,OFF-BI-10003527,Fellowes PB500 Electric Punch Plastic Comb Bin...,27453.3840
2,TEC-MA-10002412,Cisco TelePresence System EX90 Videoconferenci...,22638.4800
3,FUR-CH-10002024,HON 5400 Series Task Chairs for Big and Tall,21870.5760
4,OFF-BI-10001359,GBC DocuBind TL300 Electric Binding System,19823.4790
5,OFF-BI-10000545,GBC Ibimaster 500 Manual ProClick Binding System,19024.5000
6,TEC-CO-10001449,Hewlett Packard LaserJet 3310 Copier,18839.6860
7,TEC-MA-10001127,HP Designjet T520 Inkjet Large Format Printer ...,18374.8950
8,OFF-BI-10004995,GBC DocuBind P400 Electric Binding System,17965.0680
9,OFF-SU-10000151,High Speed Automatic Electric Letter Opener,17030.3120


## Part D — Common Table Expressions (CTEs)
**Aim:** Make complex logic readable and testable in steps.

### D1. Multi‑step ranking with CTEs
**Prompt:**
```
BigQuery SQL only.
Goal: Within each `Region`, rank states by total sales and return top 3 per region.
Table: `[YOUR_PROJECT].superstore_data.sales`
CTE 1 (`state_sales`): SUM(Sales) by `Region`, `State`
CTE 2 (`ranked_state_sales`): Add `RANK() OVER (PARTITION BY Region ORDER BY total_sales DESC)` as `sales_rank`
Final SELECT: rows where `sales_rank <= 3`
Output columns: `Region`, `State`, `total_sales`, `sales_rank`
Sort: by `Region`, then `sales_rank`
```
**Ask for**: a one-paragraph explanation of each step, then **provide only the final runnable SQL**.

This query works in three stages to ensure a clear and accurate result. The first CTE, `state_sales`, aggregates the raw data by summing up all sales for each unique combination of `Region` and `State`, creating a summary table of total sales. The second CTE, `ranked_state_sales`, then takes this summary and applies the `RANK()` window function, which calculates a sales rank for each state *within* its own region without collapsing the data further. Finally, the main `SELECT` statement filters these ranked results to only include rows where the rank is 3 or less, effectively giving us the top three states for each region, sorted for clean presentation.

In [ ]:
query_string = """
WITH state_sales AS (
  -- First, aggregate the total sales for each state in each region
  SELECT
    Region,
    State,
    SUM(Sales) AS total_sales
  FROM
    `mgmt-467-lab-1-ai-assisted-sql.lab1_foundation.superstore_clean`
  GROUP BY
    Region, State
),
ranked_state_sales AS (
  -- Then, rank each state within its region based on total sales
  SELECT
    *,
    RANK() OVER (PARTITION BY Region ORDER BY total_sales DESC) as sales_rank
  FROM
    state_sales
)
-- Finally, select only the top 3 states from each region
SELECT
  Region,
  State,
  total_sales,
  sales_rank
FROM
  ranked_state_sales
WHERE
  sales_rank <= 3
ORDER BY
  Region,
  sales_rank;
"""
query_job = client.query(query_string)
results_df = query_job.to_dataframe()
display(results_df)

,Region,State,total_sales,sales_rank
0,Central,Texas,170188.0458,1
1,Central,Illinois,80166.1010,2
2,Central,Michigan,76269.6140,3
3,East,New York,310876.2710,1
4,East,Pennsylvania,116511.9140,2
5,East,Ohio,78258.1360,3
6,South,Florida,89473.7080,1
7,South,Virginia,70636.7200,2
8,South,North Carolina,55603.1640,3
9,West,California,457687.6315,1


### D2. Time‑boxed “most improved” analysis
**Prompt:**
```
BigQuery SQL only.
Goal: Identify the top 5 sub-categories with the largest YoY revenue increase from 2023 to 2024.
Table: `[YOUR_PROJECT].superstore_data.sales`
CTE `yr_sales`: SUM(Sales) by `Sub_Category` and `year` extracted from `Order_Date`
Final: pivot or self-join to compute delta (2024 minus 2023) as `yoy_delta`
Output: `Sub_Category`, `sales_2023`, `sales_2024`, `yoy_delta`
Order by `yoy_delta` DESC
Limit 5
```
**Validation:** Ask the model for two quick failure modes (e.g., missing years) and how to handle them.

In [ ]:
query_string = """
WITH yr_sales AS (
  SELECT
    sub_category,
    SUM(CASE WHEN EXTRACT(YEAR FROM order_date) = 2016 THEN sales ELSE 0 END) AS sales_2016,
    SUM(CASE WHEN EXTRACT(YEAR FROM order_date) = 2017 THEN sales ELSE 0 END) AS sales_2017
  FROM
    `mgmt-467-lab-1-ai-assisted-sql.lab1_foundation.superstore_clean`
  WHERE
    EXTRACT(YEAR FROM order_date) IN (2016, 2017)
  GROUP BY
    sub_category
)
SELECT
  sub_category,
  sales_2016,
  sales_2017,
  (sales_2017 - sales_2016) AS yoy_delta
FROM
  yr_sales
ORDER BY
  yoy_delta DESC
LIMIT 5;
"""

query_job = client.query(query_string)
results_df = query_job.to_dataframe()
display(results_df)

,sub_category,sales_2016,sales_2017,yoy_delta
0,Phones,78962.030,105340.516,26378.486
1,Binders,49683.325,72788.045,23104.720
2,Accessories,41895.854,59946.232,18050.378
3,Appliances,26050.315,42926.932,16876.617
4,Copiers,49599.410,62899.388,13299.978


Failure Mode 1: Sub-Category Exists in Only One Year

Problem: A sub-category might have sales in 2017 but not in 2016 (a new product line), or vice-versa (a discontinued line). The current query would treat the missing year's sales as zero, leading to a misleadingly large positive or negative yoy_delta. A new product line with $10,000 in sales would show a $$10,000 in sales would show a $10,000 'increase', which isn't a true year-over-year comparison.
How to Handle: To ensure you're comparing established products, you can add a HAVING clause to filter for sub-categories that have sales in both years. This makes the comparison more direct. For example:
HAVING SUM(CASE WHEN EXTRACT(YEAR FROM order_date) = 2016 THEN sales ELSE 0 END) > 0
   AND SUM(CASE WHEN EXTRACT(YEAR FROM order_date) = 2017 THEN sales ELSE 0 END) > 0
Failure Mode 2: Comparing Incomplete Years

Problem: If your data for the most recent year (e.g., 2017) only runs through June, you would be incorrectly comparing a full 12 months of 2016 data against only 6 months of 2017 data. This would make growth for nearly every category appear negative.
How to Handle: Before running the analysis, always check the maximum date in your dataset. If a year is incomplete, you should adjust the logic to compare equivalent time periods. For instance, compare sales from Jan 1 - Jun 30 of 2016 against sales from Jan 1 - Jun 30 of 2017.

## Part E — Window Functions (ROW_NUMBER, RANK, DENSE_RANK, LAG/LEAD, moving averages)
**Aim:** Compare rows across partitions and time; compute trends and ranks without collapsing rows.

### E1. Top product per region (ROW_NUMBER)
**Prompt:**
```
BigQuery SQL only.
Task: For each `Region`, return only the single highest-revenue `Sub_Category`.
Table: `[YOUR_PROJECT].superstore_data.sales`
CTE `subcat_sales`: SUM(Sales) by `Region`, `Sub_Category`
Add `ROW_NUMBER() OVER (PARTITION BY Region ORDER BY total_sales DESC)` as rn
Final: filter `rn = 1`
Output: `Region`, `Sub_Category`, `total_sales`
Sort by `Region`
```
**Why `ROW_NUMBER` instead of `RANK`?** Ask the model to add a 2-sentence contrast.

In [ ]:
query_string = """
WITH subcat_sales AS (
  -- First, calculate total sales for each sub-category within each region
  SELECT
    Region,
    sub_category,
    SUM(Sales) AS total_sales
  FROM
    `mgmt-467-lab-1-ai-assisted-sql.lab1_foundation.superstore_clean`
  GROUP BY
    Region, sub_category
),
ranked_sales AS (
  -- Now, rank each sub-category within its region based on total sales
  SELECT
    Region,
    sub_category,
    total_sales,
    ROW_NUMBER() OVER (PARTITION BY Region ORDER BY total_sales DESC) as rn
  FROM
    subcat_sales
)
-- Finally, select only the top-ranked sub-category from each region
SELECT
  Region,
  sub_category,
  total_sales
FROM
  ranked_sales
WHERE
  rn = 1
ORDER BY
  Region;
"""

# Execute the query and display the results
query_job = client.query(query_string)
results_df = query_job.to_dataframe()
display(results_df)

,Region,sub_category,total_sales
0,Central,Chairs,85230.646
1,East,Phones,100614.982
2,South,Phones,58304.438
3,West,Chairs,101781.328


ROW_NUMBER() assigns a unique number to every row, so it will always pick exactly one row as number 1, even if there's a tie in sales. In contrast, RANK() gives tied rows the same rank, meaning if two sub-categories had the exact same top sales, they would both be ranked 1 and the query would return multiple rows for that region.

### E2. YoY growth with LAG
**Prompt:**
```
BigQuery SQL only.
Task: Compute year-over-year revenue growth for 'Phones' sub-category.
Table: `[YOUR_PROJECT].superstore_data.sales`
Steps:
- Filter to `Sub_Category = 'Phones'`
- Aggregate yearly revenue using EXTRACT(YEAR FROM Order_Date)
- Add `LAG(yearly_revenue) OVER (ORDER BY year)` as `prev_revenue`
- Compute `yoy_pct = 100.0 * (yearly_revenue - prev_revenue) / prev_revenue`
Output: `year`, `yearly_revenue`, `prev_revenue`, `yoy_pct`
Sort by `year` ASC
```
**Ask for**: a guard against divide-by-zero or NULL previous year.

In [ ]:
query_string = """
WITH yearly_phone_sales AS (
  -- Step 1 & 2: Filter for 'Phones' and aggregate sales by year
  SELECT
    EXTRACT(YEAR FROM order_date) AS year,
    SUM(sales) AS yearly_revenue
  FROM
    `mgmt-467-lab-1-ai-assisted-sql.lab1_foundation.superstore_clean`
  WHERE
    sub_category = 'Phones'
  GROUP BY
    year
),
phone_sales_with_lag AS (
  -- Step 3: Use LAG to get the previous year's revenue
  SELECT
    year,
    yearly_revenue,
    LAG(yearly_revenue, 1) OVER (ORDER BY year) AS prev_revenue
  FROM
    yearly_phone_sales
)
-- Final Step: Calculate YoY percentage growth with a divide-by-zero guard
SELECT
  year,
  yearly_revenue,
  prev_revenue,
  -- Guard against division by zero for the first year
  ROUND(100.0 * (yearly_revenue - prev_revenue) / NULLIF(prev_revenue, 0), 2) AS yoy_pct
FROM
  phone_sales_with_lag
ORDER BY
  year ASC;
"""

# Execute the query and display the results
query_job = client.query(query_string)
results_df = query_job.to_dataframe()
display(results_df)

,year,yearly_revenue,prev_revenue,yoy_pct
0,2014,77390.806,NaN,NaN
1,2015,68313.702,77390.806,-11.73
2,2016,78962.030,68313.702,15.59
3,2017,105340.516,78962.030,33.41


### E3. 3‑month moving average (MA)
**Prompt:**
```
BigQuery SQL only.
Task: For the 'Corporate' segment, compute a 3-month moving average of monthly revenue.
Table: `[YOUR_PROJECT].superstore_data.sales`
Steps:
- Derive `month` via DATE_TRUNC(Order_Date, MONTH)
- SUM(Sales) per `month`
- Add `AVG(monthly_revenue) OVER (ORDER BY month ROWS BETWEEN 2 PRECEDING AND CURRENT ROW)` as `ma_3`
Output: `month`, `monthly_revenue`, `ma_3`
Sort by `month` ASC
```
**Tip:** Ask the model to include a 1‑line cost control note (e.g., restrict date range while iterating).

In [ ]:
# Cost Control Note: During development, it's wise to add a date filter to the
# initial CTE (e.g., `WHERE segment = 'Corporate' AND EXTRACT(YEAR FROM order_date) > 2015`)
# to reduce the amount of data scanned and speed up iterations.

query_string = """
WITH monthly_corporate_sales AS (
  -- Filter for the 'Corporate' segment and calculate total monthly revenue
  SELECT
    DATE_TRUNC(order_date, MONTH) AS month,
    SUM(sales) AS monthly_revenue
  FROM
    `mgmt-467-lab-1-ai-assisted-sql.lab1_foundation.superstore_clean`
  WHERE
    segment = 'Corporate'
  GROUP BY
    month
)
-- Calculate the 3-month moving average of revenue
SELECT
  month,
  monthly_revenue,
  ROUND(AVG(monthly_revenue) OVER (
    ORDER BY month
    ROWS BETWEEN 2 PRECEDING AND CURRENT ROW
  ), 2) AS ma_3
FROM
  monthly_corporate_sales
ORDER BY
  month ASC;
"""

# Execute the query and display the results
query_job = client.query(query_string)
results_df = query_job.to_dataframe()
display(results_df)

,month,monthly_revenue,ma_3
0,2014-01-01,1701.5280,1701.53
1,2014-02-01,1183.6680,1442.60
2,2014-03-01,11106.7990,4664.00
3,2014-04-01,14131.7290,8807.40
4,2014-05-01,9142.0000,11460.18
5,2014-06-01,3970.9140,9081.55
6,2014-07-01,10032.9880,7715.30
7,2014-08-01,7451.7740,7151.89
8,2014-09-01,15507.7450,10997.50
9,2014-10-01,12637.6780,11865.73


## Part F — Debugging & Optimization Prompts
**Aim:** Use the model as a rubber duck for error handling and performance.

### F1. Explain the error, propose a fix
**Prompt:**
```
I ran this BigQuery SQL and got an error:
WITH yr_sales AS (
  SELECT
    sub_category,
    SUM(CASE WHEN EXTRACT(YEAR FROM order_date) = 2016 THEN sales ELSE 0 END) AS sales_2016,
    SUM(CASE WHEN EXTRACT(YEAR FROM order_date) = 2017 THEN sales ELSE 0 END) AS sales_2017
  FROM
    `mgmt-467-lab-1-ai-assisted-sql.lab1_foundation.superstore_clean`
  WHERE
    EXTRACT(YEAR FROM order_date) IN (2016, 2017)
  GROUP BY
    sub_category
)
SELECT
  sub_category,
  sales_2016,
  sales_2017,
  (sales_2017 - sales_2016) AS yoy_delta
FROM
  yr_sales
ORDER BY
  yoy_delta DESC
LIMIT 5;
Act as a BigQuery trouble‑shooter.
1) Identify the root cause.
2) Propose the smallest possible fix.
3) Suggest a quick sanity check query to verify the fix.
Return only the corrected SQL and a 2‑sentence rationale.
```

The provided query is syntactically correct, but its logic can be misleading by including sub-categories with no sales in 2016, making their growth from zero appear artificially large. The corrected query adds a `HAVING` clause to ensure a true year-over-year comparison is only made on sub-categories that had sales in the earlier period.

```sql
-- Corrected SQL
WITH yr_sales AS (
  SELECT
    sub_category,
    SUM(CASE WHEN EXTRACT(YEAR FROM order_date) = 2016 THEN sales ELSE 0 END) AS sales_2016,
    SUM(CASE WHEN EXTRACT(YEAR FROM order_date) = 2017 THEN sales ELSE 0 END) AS sales_2017
  FROM
    `mgmt-467-lab-1-ai-assisted-sql.lab1_foundation.superstore_clean`
  WHERE
    EXTRACT(YEAR FROM order_date) IN (2016, 2017)
  GROUP BY
    sub_category
  -- The fix: Ensure products existed in the first year to avoid misleading "growth from zero"
  HAVING sales_2016 > 0
)
SELECT
  sub_category,
  sales_2016,
  sales_2017,
  (sales_2017 - sales_2016) AS yoy_delta
FROM
  yr_sales
ORDER BY
  yoy_delta DESC
LIMIT 5;
```

### F2. Reduce cost / improve speed
**Prompt:**
```
Act as a BigQuery cost optimizer.
Given this query (below), list 3 ways to reduce scanned bytes and improve performance without changing the business logic.

WITH yearly_phone_sales AS (
  -- Step 1 & 2: Filter for 'Phones' and aggregate sales by year
  SELECT
    EXTRACT(YEAR FROM order_date) AS year,
    SUM(sales) AS yearly_revenue
  FROM
    `mgmt-467-lab-1-ai-assisted-sql.lab1_foundation.superstore_clean`
  WHERE
    sub_category = 'Phones'
  GROUP BY
    year
),
phone_sales_with_lag AS (
  -- Step 3: Use LAG to get the previous year's revenue
  SELECT
    year,
    yearly_revenue,
    LAG(yearly_revenue, 1) OVER (ORDER BY year) AS prev_revenue
  FROM
    yearly_phone_sales
)
-- Final Step: Calculate YoY percentage growth with a divide-by-zero guard
SELECT
  year,
  yearly_revenue,
  prev_revenue,
  -- Guard against division by zero for the first year
  ROUND(100.0 * (yearly_revenue - prev_revenue) / NULLIF(prev_revenue, 0), 2) AS yoy_pct
FROM
  phone_sales_with_lag
ORDER BY
  year ASC;
  
Act as a BigQuery trouble‑shooter.
1) Identify the root cause.
2) Propose the smallest possible fix.
3) Suggest a quick sanity check query to verify the fix.
Return only the corrected SQL and a 2‑sentence rationale.
Prioritize: partition filters, column pruning, pre-aggregations, and temporary results via CTEs.
```

Table Partitioning & Clustering: Partition the superstore_clean table by order_date and cluster it by sub_category. This is the most impactful optimization for this query.
Explicit Date Filtering (Conditional): If the analysis is only for a specific year range, add an EXTRACT(YEAR FROM order_date) BETWEEN start_year AND end_year to the WHERE clause to leverage partition pruning.
Leverage Columnar Storage: Be aware that BigQuery reads only referenced columns. The current query is already structured to only reference order_date, sales, and sub_category for its initial scan, which is efficient.

```sql
WITH yearly_phone_sales AS (
  -- Step 1 & 2: Filter for 'Phones' and aggregate sales by year
  SELECT
    EXTRACT(YEAR FROM order_date) AS year,
    SUM(sales) AS yearly_revenue
  FROM
    `mgmt-467-lab-1-ai-assisted-sql.lab1_foundation.superstore_clean`
  WHERE
    sub_category = 'Phones'
  GROUP BY
    year
),
phone_sales_with_lag AS (
  -- Step 3: Use LAG to get the previous year's revenue
  SELECT
    year,
    yearly_revenue,
    LAG(yearly_revenue, 1) OVER (ORDER BY year) AS prev_revenue
  FROM
    yearly_phone_sales
)
-- Final Step: Calculate YoY percentage growth with a divide-by-zero guard
SELECT
  year,
  yearly_revenue,
  prev_revenue,
  -- Guard against division by zero for the first year
  ROUND(100.0 * (yearly_revenue - prev_revenue) / NULLIF(prev_revenue, 0), 2) AS yoy_pct
FROM
  phone_sales_with_lag
ORDER BY
  year ASC;
```

**Root Cause:** The query scans all data in `superstore_clean` to filter for `sub_category = 'Phones'` and extract the year from `order_date`.
**Smallest Possible Fix:** Partition `superstore_clean` by `order_date` and cluster it by `sub_category`. The query is already leveraging column pruning via CTEs by selecting only necessary columns for aggregation.
**Sanity Check Query:** This query is already well-formed; verifying the fix involves checking the BigQuery job details after the table optimizations are applied to confirm reduced bytes scanned and faster execution.

---
```sql
WITH yearly_phone_sales AS (
  -- Step 1 & 2: Filter for 'Phones' and aggregate sales by year
  SELECT
    EXTRACT(YEAR FROM order_date) AS year,
    SUM(sales) AS yearly_revenue
  FROM
    `mgmt-467-lab-1-ai-assisted-sql.lab1_foundation.superstore_clean`
  WHERE
    -- Assuming this table is very large and order_date has significant yearly variation,
    -- adding a date filter here will prune partitions if the table is partitioned by order_date.
    -- If you need all historical data, this filter might not be applicable unless the
    -- date range for 'Phones' is known to be smaller than the entire table.
    -- For demonstration, we'll assume a reasonable recent date range.
    order_date BETWEEN '2010-01-01' AND '2023-12-31' -- Example: Adjust based on known data range or business needs
    AND sub_category = 'Phones'
  GROUP BY
    year
),
phone_sales_with_lag AS (
  -- Step 3: Use LAG to get the previous year's revenue
  SELECT
    year,
    yearly_revenue,
    LAG(yearly_revenue, 1) OVER (ORDER BY year) AS prev_revenue
  FROM
    yearly_phone_sales
)
-- Final Step: Calculate YoY percentage growth with a divide-by-zero guard
SELECT
  year,
  yearly_revenue,
  prev_revenue,
  -- Guard against division by zero for the first year
  ROUND(100.0 * (yearly_revenue - prev_revenue) / NULLIF(prev_revenue, 0), 2) AS yoy_pct
FROM
  phone_sales_with_lag
ORDER BY
  year ASC;
```
Adding an explicit date range filter to the `WHERE` clause will significantly reduce scanned bytes by pruning partitions if the `superstore_clean` table is partitioned by `order_date`. This ensures BigQuery only processes data from relevant time periods, even if the primary filter is `sub_category`.

**Sanity Check Query:**
```sql
SELECT
  EXTRACT(YEAR FROM order_date) AS year,
  sub_category,
  COUNT(*) AS row_count
FROM
  `mgmt-467-lab-1-ai-assisted-sql.lab1_foundation.superstore_clean`
WHERE
  -- Use the same date range as in the optimized query
  order_date BETWEEN '2010-01-01' AND '2023-12-31'
  AND sub_category = 'Phones'
GROUP BY
  year,
  sub_category
ORDER BY
  year;
```
---

```sql
WITH yearly_phone_sales AS (
  -- Step 1 & 2: Filter for 'Phones' and aggregate sales by year
  SELECT
    EXTRACT(YEAR FROM order_date) AS year,
    SUM(sales) AS yearly_revenue
  FROM
    `mgmt-467-lab-1-ai-assisted-sql.lab1_foundation.superstore_clean`
  WHERE
    sub_category = 'Phones'
  GROUP BY
    year
),
phone_sales_with_lag AS (
  -- Step 3: Use LAG to get the previous year's revenue
  SELECT
    year,
    yearly_revenue,
    LAG(yearly_revenue, 1) OVER (ORDER BY year) AS prev_revenue
  FROM
    yearly_phone_sales
)
-- Final Step: Calculate YoY percentage growth with a divide-by-zero guard
SELECT
  year,
  yearly_revenue,
  prev_revenue,
  -- Guard against division by zero for the first year
  ROUND(100.0 * (yearly_revenue - prev_revenue) / NULLIF(prev_revenue, 0), 2) AS yoy_pct
FROM
  phone_sales_with_lag
ORDER BY
  year ASC;
```
The original query implicitly uses column pruning by selecting only the necessary columns (`order_date`, `sales`, `sub_category`) for its aggregation. This is BigQuery's default columnar storage behavior.
The primary performance bottleneck is the scan of all rows in `superstore_clean` before filtering by `sub_category`.

**Sanity Check Query:**
```sql
SELECT
  sub_category,
  COUNT(*) AS row_count
FROM
  `mgmt-467-lab-1-ai-assisted-sql.lab1_foundation.superstore_clean`
WHERE
  sub_category = 'Phones'
LIMIT 10;
```

## Part G — Validation & Counter‑examples (DIVE: Validate)
**Aim:** Avoid “first‑answer fallacy” by testing alternatives.

### G1. Ask for counter‑queries
**Prompt:**
```
I concluded that 'Tables' is a high‑sales but negative‑profit sub-category due to high discounts.
Create two alternative BigQuery SQL queries that could falsify or nuance this finding:
- One that slices by region and time
- One that controls for order priority or ship mode
Return BigQuery SQL only, then a one-paragraph note on how to compare outcomes.
```

In [ ]:
#-- Alternative 1: Slicing by Region and Time
query_string = """
SELECT
  region,
  EXTRACT(YEAR FROM order_date) AS order_year,
  sub_category,
  SUM(sales) AS total_sales,
  SUM(profit) AS total_profit,
  SUM(discount) AS total_discount
FROM
  `mgmt-467-lab-1-ai-assisted-sql.lab1_foundation.superstore_clean`
WHERE
  sub_category = 'Tables'
GROUP BY
  region,
  order_year,
  sub_category
ORDER BY
  region,
  order_year;

"""

# Execute the query and display the results
query_job = client.query(query_string)
results_df = query_job.to_dataframe()
display(results_df)

,region,order_year,sub_category,total_sales,total_profit,total_discount
0,Central,2014,Tables,7785.4780,-1424.3310,4.90
1,Central,2015,Tables,6857.2600,-265.0939,2.90
2,Central,2016,Tables,13922.9260,292.6211,3.50
3,Central,2017,Tables,10589.3070,-2162.8466,7.60
4,East,2014,Tables,10603.7040,-3537.8375,7.60
5,East,2015,Tables,8884.8060,-2275.8641,5.60
6,East,2016,Tables,7825.3280,-2306.7783,8.10
7,East,2017,Tables,11825.9690,-2904.9002,8.60
8,South,2014,Tables,9940.9445,1107.9902,1.25
9,South,2015,Tables,7370.6745,-2171.3765,1.75


```sql
-- Alternative 1: Slicing by Region and Time
SELECT
  region,
  EXTRACT(YEAR FROM order_date) AS order_year,
  sub_category,
  SUM(sales) AS total_sales,
  SUM(profit) AS total_profit,
  SUM(discount) AS total_discount
FROM
  `mgmt-467-lab-1-ai-assisted-sql.lab1_foundation.superstore_clean`
WHERE
  sub_category = 'Tables'
GROUP BY
  region,
  order_year,
  sub_category
ORDER BY
  region,
  order_year;
```

```sql
-- Alternative 2: Controlling for Order Priority or Ship Mode
SELECT
  order_priority, -- Or SHIP_MODE if you prefer to slice by ship mode
  sub_category,
  SUM(sales) AS total_sales,
  SUM(profit) AS total_profit,
  SUM(discount) AS total_discount
FROM
  `mgmt-467-lab-1-ai-assisted-sql.lab1_foundation.superstore_clean`
WHERE
  sub_category = 'Tables'
GROUP BY
  order_priority, -- Or SHIP_MODE
  sub_category
ORDER BY
  order_priority; -- Or SHIP_MODE
```

To compare the outcomes of these alternative queries against your initial finding, examine the results for the 'Tables' sub-category. For the first query, look for specific regions or years where 'Tables' might actually be profitable or have lower-than-expected losses, which would nuance your initial conclusion. For the second query, observe if particular order priorities or ship modes associated with 'Tables' show profitability or less severe losses, suggesting that the high discount impact might be concentrated in specific operational contexts, rather than being a uniform issue across all 'Tables' sales.

## Part H — Synthesis (DIVE: Extend)
**Aim:** Turn analysis into business‑ready insights.

### H1. Executive‑style summary
**Prompt:**
```
Act as a business strategist.
Based on the following metrics/figures (briefly summarize your results here), write a 4-sentence executive summary:
- 1 sentence: what changed and by how much
- 1 sentence: why it likely changed (drivers)
- 1 sentence: recommended action (who/what/when)
- 1 sentence: metric to monitor next
```

Based on our analysis, the 'Phones' sub-category experienced a significant YoY revenue increase of 33.41% from 2016 to 2017, making it a key growth driver. This surge was likely influenced by factors such as new product releases, targeted marketing campaigns, or favorable market trends in the technology sector. We recommend that the sales team prioritize 'Phones' in Q4 2017 to capitalize on this momentum and further boost revenue. The key metric to monitor is the monthly sales growth rate for 'Phones' to ensure sustained performance.

### H2. Convert final SQL into an automated job (optional)
**Prompt (use only after your SQL is final):**
```
Convert my final BigQuery SQL into a Python script that can run as a scheduled job from Colab or Cloud Functions.
Requirements:
- Use python‑bigquery client
- Parameterize date range
- Write results to a destination table `[YOUR_PROJECT].analytics.outputs_kpi`
- Add basic error handling & logging
Return one complete runnable script.
```

---
## Submission checklist
- [ ] Kept prompts precise and reproducible  
- [ ] Captured at least **one** CTE query and **one** window function query  
- [ ] Documented **two** validation attempts (counter‑queries or alternate slice)  
- [ ] Wrote a 4‑sentence executive summary based on results  
- [ ] (Optional) Converted final query into a scheduled job
---